In [58]:
import os
from keras.models import load_model
model = load_model("./Models/3.model")

In [59]:
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

In [60]:
def getContours(img,imgContour):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    new=[]
    for cnt in contours:
        area = cv2.contourArea(cnt)
        areaMin = 5000
        if area > areaMin:
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            print(len(approx))
            x , y , w, h = cv2.boundingRect(approx)
           
            if h>=50 and w>=50 and h-w<10:
                 cv2.rectangle(imgContour, (x-50 , y-50 ), (x + w+50 , y + h +50), (0, 255, 0), 5)
                 new.append(((x , y ), (x + w , y + h )))
    return new


In [61]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt

#image
images = os.listdir('./images')
for image in images:
    data=[]
    path = "1.jpg"
    image = cv2.imread(path)
    img=image
    imgContour = img.copy()
    imgBlur = cv2.GaussianBlur(img, (7, 7), 1)
    imgGray = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2GRAY)
    threshold1 =255
    threshold2 = 255
    imgCanny = cv2.Canny(imgGray,threshold1,threshold2)
    kernel = np.ones((5, 5))
    imgDil = cv2.dilate(imgCanny, kernel, iterations=1)
    array = getContours(imgDil,imgContour)
    print(array)
    for cropped in range(len(array)):
        ROI = image[array[0][0][1]-50:array[0][1][1]+50,array[0][0][0]-50:array[0][1][0]+50]
        cv2.imshow('roi',ROI)
        resize = cv2.resize(ROI,(50,50))
        data.append(np.array(resize))
        X_test=np.array(data)
        Y_pred = model.predict_classes(X_test)
        print("Predicted traffic sign is: ", classes[Y_pred[0]])
        cv2.putText(imgContour,  classes[Y_pred[0]] , (10,50), cv2.FONT_HERSHEY_COMPLEX, 2,
                        (0, 0, 0), 2)
    cv2.imshow('image',cv2.resize(imgContour,(500,500)))
    k = cv2.waitKey(0) & 0xff
    if k == 27:
        break

#video
'''
cap= cv2.VideoCapture('2.mp4')
temp=50
while(1):
    ret, frame = cap.read()
    resized = cv2.resize(frame,(50,50))
    data = []
    data.append(np.array(resized))
    X_test = np.array(data)
    Y_pred = model.predict_classes(X_test)
    #print(temp-Y_pred)
    if(temp-Y_pred[0]):
        print("Traffic sign : ", classes[Y_pred[0]])
    temp=Y_pred[0]
    cv2.imshow('video', cv2.resize(frame,(300,300))
    cap.release()'''
cv2.destroyAllWindows()

2
14
[((160, 493), (296, 633))]
Predicted traffic sign is:  Speed limit (100km/h)
